# Prepare Models

#### Create PyTorch models from timm

In [ ]:
import timm

model_names = ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152"]
models = [timm.create_model(m) for m in model_names]

#### Convert PyTorch models to OpenVINO

In [ ]:
from pathlib import Path

import nncf
import torch
from torch.utils.data import DataLoader, TensorDataset
import openvino as ov
from openvino.tools import mo

# Use random tensor for calibration dataset
dataloader = DataLoader(TensorDataset(torch.randn(1024, 3, 224, 224)))
dataset = nncf.Dataset(dataloader, lambda item: item[0].numpy())

for model, name in zip(models, model_names):
    print(f"Convert {name} to OpenVINO")
    ov_model = mo.convert_model(model, input_shape=[-1, 3, 224, 224])
    
    Path(f"models/{name}/fp32").mkdir(parents=True, exist_ok=True)
    Path(f"models/{name}/fp16").mkdir(parents=True, exist_ok=True)
    Path(f"models/{name}/int8").mkdir(parents=True, exist_ok=True)

    ov.save_model(ov_model, f"models/{name}/fp32/model.xml", compress_to_fp16=False)
    ov.save_model(ov_model, f"models/{name}/fp16/model.xml", compress_to_fp16=True)
    ov_model_int8 = nncf.quantize(ov_model, dataset, subset_size=len(dataloader))
    ov.save_model(ov_model_int8, f"models/{name}/int8/model.xml")